# Inside Airbnb Data - Part 1: Entity Resolution

## Loading Libraries & Data

In [1]:
# Importing Libraries
import os 
import pandas as pd
import polars as pl
import duckdb as ddb
from functools import reduce
from IPython.display import display
pd.set_option('display.max_columns', None)

### Creating DuckDB File

In [2]:
# Pointing towards the directory of this file
notebook_folder = os.getcwd()

# Pointing towards the directory of the data file
data_folder = os.path.join(notebook_folder, "..", "data")
csvs = []

# Looping through CSV files in the data folder of the repo
for file in os.listdir(data_folder):
    if file.endswith(".csv"):
        csvs.append(os.path.join(data_folder, file))

# Placing file in the data folder
duckdb_file = os.path.join(data_folder, "airbnb_listings.duckdb")

# DuckDB connection
con = ddb.connect(duckdb_file)

# Looping through each CSV file to create a table out of each CSV and remove .csv file extension
for file in csvs:
    table_name = os.path.splitext(os.path.basename(file))[0]
    table_name = str(table_name).lower()
    file_path = file.replace("\\", "/") # Because Windows slashes...
    
    con.execute(f"""
    CREATE OR REPLACE Table "{table_name}" AS 
    SELECT * 
    FROM read_csv_auto('{file_path}')
    """)

### Loading DuckDB Tables

In [3]:
# DataFrame of DuckDB table names
tables_dfs = con.execute("SHOW TABLES").df()

In [4]:
# Making list of table names
table_names = tables_dfs["name"].to_list()

In [5]:
# Loading each table as a dictionary of Polars DataFrames to avoid loading a dozen CSVs into memory
tables_dict = {}
for table in table_names:
    df = con.execute(f"SELECT * FROM {table}").pl()
    tables_dict[table] = df

In [6]:
# Creating a DataFrame containing the number of rows and columns in each table
summary_list = []

for table_name, df in tables_dict.items():
    n_rows, n_cols = df.shape
    summary_list.append({
        "table_name": table_name,
        "row_count": n_rows,
        "col_count": n_cols
        })

table_shapes = pl.DataFrame(summary_list)

In [7]:
table_shapes.to_pandas()

,table_name,row_count,col_count
0,apr_2025_listings,37147,79
1,aug_2025_listings,36403,79
2,dec_2024_listings,37808,75
3,feb_2025_listings,37800,79
4,jan_2025_listings,37784,75
5,jul_2025_listings,36345,79
6,jun_2025_listings,36322,79
7,mar_2025_listings,37434,79
8,may_2025_listings,37018,79
9,neighbourhoods,230,2


In [8]:
# Removing neighborhoods table
tables_dict.pop("neighbourhoods", None)
tables_dict.keys()

dict_keys(['apr_2025_listings', 'aug_2025_listings', 'dec_2024_listings', 'feb_2025_listings', 'jan_2025_listings', 'jul_2025_listings', 'jun_2025_listings', 'mar_2025_listings', 'may_2025_listings', 'nov_2024_listings', 'oct_2024_listings'])

## Column Name Comparisons

In [9]:
# Finding columns in common using set intersection
cols_intersection = reduce(lambda a, b: a & b, (set(df.columns) for df in tables_dict.values()))

# Finding all columns with set unions
cols_union = reduce(lambda a, b: a | b, (set(df.columns) for df in tables_dict.values()))

# Finding columns not found in every table
diff_cols = cols_union - cols_intersection
len(diff_cols)

4

In [10]:
diff_cols

{'availability_eoy',
 'estimated_occupancy_l365d',
 'estimated_revenue_l365d',
 'number_of_reviews_ly'}

In [11]:
# Removing columns from all tables in diff_cols
for table_name, df in tables_dict.items():
    tables_dict[table_name] = df.select([col for col in df.columns if col not in diff_cols])

In [12]:
# Checking again for column differences
cols_intersection = reduce(lambda a, b: a & b, (set(df.columns) for df in tables_dict.values()))
cols_union = reduce(lambda a, b: a | b, (set(df.columns) for df in tables_dict.values()))
diff_cols = cols_union - cols_intersection
len(diff_cols)

0

## Joining Datasets

In [13]:
# Ensuring all tables have the same column order for vertical concatenation
for table_name, df in tables_dict.items():
    tables_dict[table_name] = df.select(sorted(cols_union))

In [14]:
# Picking a reference DataFrame and making its schema a reference to cast dtypes to all tables for vertical concatenation
reference_df = tables_dict["aug_2025_listings"] # Most recent table as of this writing

reference_schema = {}
for col in reference_df.columns:
    reference_schema[col] = reference_df[col].dtype

In [15]:
# Checking for mismatched data types across dataframes
mismatched_columns = {}

for table_name, df in tables_dict.items():
    mismatched = []
    for col in df.columns:
        target_dtype = reference_schema[col]
        if df[col].dtype != target_dtype:
            mismatched.append((col, df[col].dtype, target_dtype))
    if mismatched:
        mismatched_columns[table_name] = mismatched

# Displaying mismatched columns
for table, cols in mismatched_columns.items():
    print(f"Table: {table}")
    for col, current_dtype, expected_dtype in cols:
        print(f"  Column: {col} | Current: {current_dtype} | Expected: {expected_dtype}")

# Only A single column across 4 tables with an insignificant data type mismatch, so proceeding with casting data types.

Table: jun_2025_listings
  Column: maximum_maximum_nights | Current: Int64 | Expected: Float64
  Column: maximum_minimum_nights | Current: Int64 | Expected: Float64
  Column: minimum_maximum_nights | Current: Int64 | Expected: Float64
  Column: minimum_minimum_nights | Current: Int64 | Expected: Float64


In [16]:
# Casting the reference data types onto every table
for table_name, df in tables_dict.items():
    casted_dfs = {}
    for col in df.columns:
        target_datatype = reference_schema[col]
        if df[col].dtype != target_datatype:
            casted_dfs[col] = df[col].cast(target_datatype)
        else:
            casted_dfs[col] = df[col]
    tables_dict[table_name] = pl.DataFrame(casted_dfs)

In [17]:
# Concatenating all tables the dictionary into a single DataFrame
all_listings = pl.concat(tables_dict.values(), how = "vertical")

## Adding Concatenated Table to DuckDB File

In [18]:
# Creating view of Polars df to query with DuckDB
con.register("all_listings_view", all_listings)

# Adding Polars df of the concatenated table to the DuckDB file
con.execute("CREATE OR REPLACE TABLE all_listings AS SELECT * FROM all_listings_view")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [19]:
# Confirming the new table was added
print(con.execute("PRAGMA table_info(all_listings)").df())

    cid                 name     type  notnull dflt_value     pk
0     0         accommodates   BIGINT    False       None  False
1     1            amenities  VARCHAR    False       None  False
2     2      availability_30   BIGINT    False       None  False
3     3     availability_365   BIGINT    False       None  False
4     4      availability_60   BIGINT    False       None  False
..  ...                  ...      ...      ...        ...    ...
70   70  review_scores_value   DOUBLE    False       None  False
71   71    reviews_per_month   DOUBLE    False       None  False
72   72            room_type  VARCHAR    False       None  False
73   73            scrape_id   BIGINT    False       None  False
74   74               source  VARCHAR    False       None  False

[75 rows x 6 columns]


## Inspecting Variables

In [20]:
# Viewing columns shared across all datasets as a compact dataframe (75 columns as 5 by 15 grid)
common_cols = list(sorted(cols_intersection))

n_cols = 15
n_rows = 5
total_cells = n_cols * n_rows

cols_padded = common_cols[:total_cells] + [""] * max(0, total_cells - len(common_cols))
cols_grid = [common_cols[i * n_cols:(i + 1) * n_cols] for i in range(n_rows)]

display(pd.DataFrame(cols_grid))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bathrooms_text,bedrooms,beds,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,calendar_last_scraped
1,calendar_updated,description,first_review,has_availability,host_about,host_acceptance_rate,host_has_profile_pic,host_id,host_identity_verified,host_is_superhost,host_listings_count,host_location,host_name,host_neighbourhood,host_picture_url
2,host_response_rate,host_response_time,host_since,host_thumbnail_url,host_total_listings_count,host_url,host_verifications,id,instant_bookable,last_review,last_scraped,latitude,license,listing_url,longitude
3,maximum_maximum_nights,maximum_minimum_nights,maximum_nights,maximum_nights_avg_ntm,minimum_maximum_nights,minimum_minimum_nights,minimum_nights,minimum_nights_avg_ntm,name,neighborhood_overview,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,number_of_reviews,number_of_reviews_l30d
4,number_of_reviews_ltm,picture_url,price,property_type,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,room_type,scrape_id,source


In [21]:
# Converting to Pandas
listings = all_listings.to_pandas()

In [22]:
listings.isna().sum().sort_values(ascending = False)[:10]

calendar_updated         409009
license                  349891
neighborhood_overview    189301
neighbourhood            189290
host_about               174541
host_response_rate       168004
host_response_time       168004
price                    163836
beds                     163700
bathrooms                162449
dtype: int64

In [23]:
# Noting down potentially useless columns to inspect before removing
useless_cols = ["license", "calendar_updated", "neighborhood_overview", "host_about", "first_review", "last_review", "calendar_last_scraped", # Often null
                "listing_url", "host_url", "picture_url", "host_thumbnail_url", "host_picture_url", "source", # Urls are not useful for prediction of pricing
                "host_name", "host_location", "host_neighbourhood" # Where the host lives and their names are irrelevant for price prediction
                "neighbourhood", "neighbourhood_cleansed", "neighbourhood", "neighbourhood_group_cleaned", # redundant with lat/lon features
                "number_of_reviews_l30d", "number_of_reviews_ltm", # redundant with number_of_reviews
                "availability_60", "availability_90", # retaining only month long and year long availability - redundant
                "description" # Free text, hard to extract relevant features from for 
                "id", "scrape_id", # Unique idenitifers not needed
                "minimum_minimum_nights", "maximum_maximum_nights", "minimum_maximum_nights", "maximum_minimum_nights", # Represents values for listings beyond the time frame of the data
                "minimum_nights_avg_ntm", "maximum_nights_avg_ntm" # Aggregate values consistent for each property - low variance variables
                ] 

In [24]:
# Dropping useless columns
for col in useless_cols:
    listings.drop(columns = col, inplace = True, errors = "ignore")

In [25]:
listings.shape

(409009, 47)

In [26]:
listings.columns

Index(['accommodates', 'amenities', 'availability_30', 'availability_365',
       'bathrooms', 'bathrooms_text', 'bedrooms', 'beds',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'description',
       'has_availability', 'host_acceptance_rate', 'host_has_profile_pic',
       'host_id', 'host_identity_verified', 'host_is_superhost',
       'host_listings_count', 'host_neighbourhood', 'host_response_rate',
       'host_response_time', 'host_since', 'host_total_listings_count',
       'host_verifications', 'id', 'instant_bookable', 'last_scraped',
       'latitude', 'longitude', 'maximum_nights', 'minimum_nights', 'name',
       'neighbourhood_group_cleansed', 'number_of_reviews', 'price',
       'property_type', 'review_scores_accuracy', 'review_scores_checkin',
       'review_scores_cleanliness', 'review_scores_communication',
    

In [27]:
# Target variable is price! Removing rows will missing prices
listings = listings.dropna(subset=["price"])

In [28]:
# Viewing the shape of the dataset without null prices
listings.shape

(245173, 47)

In [29]:
listings.to_csv("listings_clean.csv", index = False)

In [30]:
# Closing connection
con.close()